In [1]:
import os
from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip
from IPython.display import Video
from open_whisper import preprocess
import subprocess

In [2]:
preprocess.download_audio(
    video_id="eh77AUKedyM",
    output_dir="/home/ubuntu/open_whisper/data/audio/eh77AUKedyM",
    ext="m4a"
)

In [3]:
preprocess.download_transcript(
    video_id="eh77AUKedyM",
    lang_code="en-US",
    output_dir="/home/ubuntu/open_whisper/scripts/notebooks",
    sub_format="vtt",
)

In [3]:
command = ["yt-dlp", "--list-formats", "https://www.youtube.com/watch?v=eh77AUKedyM"]
subprocess.run(command)

[youtube] Extracting URL: https://www.youtube.com/watch?v=eh77AUKedyM
[youtube] eh77AUKedyM: Downloading webpage
[youtube] eh77AUKedyM: Downloading ios player API JSON
[youtube] eh77AUKedyM: Downloading android player API JSON
[youtube] eh77AUKedyM: Downloading m3u8 information
[info] Available formats for eh77AUKedyM:
ID  EXT   RESOLUTION FPS CH |   FILESIZE    TBR PROTO | VCODEC           VBR ACODEC      ABR ASR MORE INFO
-------------------------------------------------------------------------------------------------------------------------
sb2 mhtml 48x27        1    |                   mhtml | images                                   storyboard
sb1 mhtml 80x45        1    |                   mhtml | images                                   storyboard
sb0 mhtml 160x90       1    |                   mhtml | images                                   storyboard
233 mp4   audio only        |                   m3u8  | audio only           unknown             [en] Default
234 mp4   audio 

CompletedProcess(args=['yt-dlp', '--list-formats', 'https://www.youtube.com/watch?v=eh77AUKedyM'], returncode=0)

In [6]:
command = ["yt-dlp", "--list-subs", "https://www.youtube.com/watch?v=jDwJawAsDbg"]
subprocess.run(command, check=True)

[youtube] Extracting URL: https://www.youtube.com/watch?v=jDwJawAsDbg
[youtube] jDwJawAsDbg: Downloading webpage
[youtube] jDwJawAsDbg: Downloading ios player API JSON
[youtube] jDwJawAsDbg: Downloading android player API JSON
[youtube] jDwJawAsDbg: Downloading m3u8 information
[info] Available automatic captions for jDwJawAsDbg:
Language      Name                                                Formats
en-GB                                                             vtt
af-en-GB      Afrikaans from English (United Kingdom)             vtt, ttml, srv3, srv2, srv1, json3
ak-en-GB      Akan from English (United Kingdom)                  vtt, ttml, srv3, srv2, srv1, json3
sq-en-GB      Albanian from English (United Kingdom)              vtt, ttml, srv3, srv2, srv1, json3
am-en-GB      Amharic from English (United Kingdom)               vtt, ttml, srv3, srv2, srv1, json3
ar-en-GB      Arabic from English (United Kingdom)                vtt, ttml, srv3, srv2, srv1, json3
hy-en-GB      Armen

CompletedProcess(args=['yt-dlp', '--list-subs', 'https://www.youtube.com/watch?v=jDwJawAsDbg'], returncode=0)

### FFMpeg - doesn't work so slay

In [23]:
transcript_file = "/home/ubuntu/open_whisper/data/transcripts/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.vtt"
audio_file = "/home/ubuntu/open_whisper/data/audio/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.m4a"

In [25]:
command = [
    "ffmpeg",
    "-f",
    "lavfi",
    "-i",
    "color=c=black:s=1280x720:r=30",
    "-i",
    f"{audio_file}",
    "-vf",
    f"subtitles={transcript_file}",
    "-c:v",
    "libx264",
    "-c:a",
    "aac",
    "/home/ubuntu/open_whisper/data/sanity-check/output.mp4",
]
subprocess.run(command)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

CompletedProcess(args=['ffmpeg', '-f', 'lavfi', '-i', 'color=c=black:s=1280x720:r=30', '-i', '/home/ubuntu/open_whisper/data/audio/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.m4a', '-vf', 'subtitles=/home/ubuntu/open_whisper/data/transcripts/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.vtt', '-c:v', 'libx264', '-c:a', 'aac', '/home/ubuntu/open_whisper/data/sanity-check/output.mp4'], returncode=1)

### MoviePy - works! so slay

In [36]:
transcript_file = "/home/ubuntu/open_whisper/data/transcripts/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.srt"
audio_file = "/home/ubuntu/open_whisper/data/audio/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.m4a"

In [37]:
audio_clip = AudioFileClip(audio_file)
audio_clip.duration

28.58

In [38]:
start_time = 0
generator = lambda text: TextClip(text, font="Helvetica", fontsize=24, color="white")
subtitles = SubtitlesClip(transcript_file, generator).set_position(("center", "bottom"))

In [39]:
video = CompositeVideoClip([subtitles], size=(800, 420)).set_audio(audio_clip)

In [40]:
video.write_videofile("/home/ubuntu/open_whisper/data/sanity-check/test.mp4", fps=24)

Moviepy - Building video /home/ubuntu/open_whisper/data/sanity-check/test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /home/ubuntu/open_whisper/data/sanity-check/test.mp4



Moviepy - Done !
Moviepy - video ready /home/ubuntu/open_whisper/data/sanity-check/test.mp4


In [41]:
Video("/home/ubuntu/open_whisper/data/sanity-check/test.mp4", width=800, height=420)